<a href="https://colab.research.google.com/github/lnpetrova/comp_ling/blob/master/ML_hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Импорты**

In [4]:
%matplotlib inline

import itertools
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mlxtend.plotting import plot_learning_curves
from mlxtend.plotting import plot_decision_regions

import numpy as np
import pandas as pd
import gensim
import string
import re
from string import punctuation
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier,  BaggingClassifier, BaggingRegressor, RandomTreesEmbedding, GradientBoostingClassifier, VotingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from mlxtend.classifier import StackingCVClassifier
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')
lem = WordNetLemmatizer()


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
!wget https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv

--2020-06-28 11:25:23--  https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4148456 (4.0M) [text/plain]
Saving to: ‘complaints.csv’

complaints.csv      100%[===================>]   3.96M  12.2MB/s    in 0.3s    

2020-06-28 11:25:25 (12.2 MB/s) - ‘complaints.csv’ saved [4148456/4148456]



In [7]:
def show_metrics(y_test, predictions):
  print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
  print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
  print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
  print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

# **Данные**

In [10]:
data = pd.read_csv('complaints.csv', sep='\t')
data.head()

,COMPLAINT_ID,DATE,PRODUCT_ID,ISSUE_ID,cleaned_text
0,3178905,03/13/2019,44,318,go year . contact advis never took loan . advi...
1,3175952,03/12/2019,44,349,"mail valid debt xx/xx/19 valid receiv , receiv..."
2,3174747,03/09/2019,44,16,xx/xx/xxxx appli receiv onlin loan bluechip fi...
3,3173291,03/08/2019,44,16,xx/xx/xxxx appli receiv onlin loan . loan amou...
4,3172221,03/07/2019,44,48,told husband left bill . debt would pay within...


In [11]:
y = data["PRODUCT_ID"]
X = data["cleaned_text"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [12]:
data.head()

,COMPLAINT_ID,DATE,PRODUCT_ID,ISSUE_ID,cleaned_text
0,3178905,03/13/2019,44,318,go year . contact advis never took loan . advi...
1,3175952,03/12/2019,44,349,"mail valid debt xx/xx/19 valid receiv , receiv..."
2,3174747,03/09/2019,44,16,xx/xx/xxxx appli receiv onlin loan bluechip fi...
3,3173291,03/08/2019,44,16,xx/xx/xxxx appli receiv onlin loan . loan amou...
4,3172221,03/07/2019,44,48,told husband left bill . debt would pay within...


# **Задание 1**

Попробуйте поднять качество классификации до 70% с помощью известных вам методов препроцессинга и Voting

In [ ]:
from sklearn import preprocessing

In [52]:
clf1 = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=1000)
clf2 = RandomForestClassifier(n_estimators=200, random_state=1)
clf3 = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth=5, 
                              min_samples_split=3, min_samples_leaf=2, max_features=100, random_state=42)
clf4 = xgb.XGBClassifier(n_estimators=150)
dtree = DecisionTreeClassifier()
bagging = BaggingClassifier(base_estimator=dtree, n_estimators=20, max_samples=0.8, max_features=0.8)

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2),
                                    ('dtc', clf3), ('xgb', clf4), ('bagging', bagging) ], voting='hard')

voting = Pipeline([
    ('vect', CountVectorizer( analyzer='word', min_df = 10, max_df = 0.7)),
    ('tfidf', TfidfTransformer(sublinear_tf=True, smooth_idf = False)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('clf', eclf),
    ])
voting = voting.fit(X_train, y_train)
predictions = voting.predict(X_test)


In [53]:
show_metrics(y_test, predictions)

Precision:   0.72
Recall:   0.71
F1-measure:   0.71
Accuracy:   0.71


# **Задание 2**

Основываясь на своих знаниях об ансамблировании различных моделей, на данных из complaints.csv достигните наилучшего результата

**Попытка 1**

In [23]:
clf1 = LogisticRegression(multi_class='multinomial', solver='lbfgs')
clf2 = RandomForestClassifier(n_estimators=150)
xgb_model = xgb.XGBClassifier(n_estimators=150)
lr = LogisticRegression()
sclf = StackingCVClassifier(classifiers=[clf1, clf2, xgb_model],
                          meta_classifier=lr, cv = 5, verbose=5)


In [ ]:
pipe = Pipeline([
                 ('feats', FeatureUnion([
                                         ('count', CountVectorizer(min_df = 7, max_df = 0.7)),
                                         ('tfidf', TfidfVectorizer(min_df = 30, max_df = 0.2))
                                         ])),
                 ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
                 ('clf', sclf)
                 ])

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
pipe = pipe.fit(X_train, y_train)

In [25]:
predictions = pipe.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.1s finished


In [26]:
show_metrics(y_test, predictions)

Precision:   0.44
Recall:   0.43
F1-measure:   0.38
Accuracy:   0.43


Результат получился хуже, попробуем другие способы

**Попытка 2**

In [34]:
clf1 = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth=5, min_samples_split=3, min_samples_leaf=2, max_features=100, random_state=42)
clf2 = RandomForestClassifier(n_estimators = 50, random_state=42)
clf3 = MultinomialNB(alpha=0.5) 
clf4 = xgb.XGBClassifier(n_estimators=150)
vot = VotingClassifier(estimators=[('dtc', clf1), ('rf', clf2), ('mn', clf3), ('xgb', clf4)], voting='hard')

In [35]:
voting = Pipeline([
    ('vect', CountVectorizer( analyzer='word', min_df = 10, max_df = 0.7)),
    ('tfidf', TfidfTransformer(sublinear_tf=True, smooth_idf = False)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('clf', vot),
    ])
voting = voting.fit(X_train, y_train)

In [36]:
predictions_voting = voting.predict(X_test)

In [37]:
show_metrics(y_test, predictions_voting)

Precision:   0.73
Recall:   0.69
F1-measure:   0.69
Accuracy:   0.69


Результаты уже близки к тем, которые были получены в задании1, точность получилась даже выше. Попробуем улучшить результаты с Voting

**Попытка 3**

In [ ]:
clf1 = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=1000)
clf2 = RandomForestClassifier(n_estimators=200, random_state=1)
clf3 = GaussianNB()


vot2 = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

voting = Pipeline([
    ('vect', CountVectorizer( analyzer='word', min_df = 10, max_df = 0.7)),
    ('tfidf', TfidfTransformer(sublinear_tf=True, smooth_idf = False)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('clf', vot2),
    ])
voting = voting.fit(X_train, y_train)
predictions = voting.predict(X_test)

In [ ]:
show_metrics(y_test, predictions)

Precision:   0.74
Recall:   0.72
F1-measure:   0.72
Accuracy:   0.72
